In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import random

In [ ]:
data = open('text.txt').read()
data = data.split('\n')
#print(len(data))
for i in data:
  if ':' in i:
    data.remove(i)
data = [element for element in data if element.strip()]
print(len(data))

for i in range(22000):
  choice = random.choice(data)
  data.remove(choice)
print(len(data))



newdata = open('textnew.txt','a')
newdata.write(str(data))
#print(newdata.read())

23508
1508


64842

In [ ]:
tokenizer = Tokenizer()

textdata= open('textnew.txt').read()

corpus = textdata.lower().split('\n')

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print(total_words)

{"'": 1, 'the': 2, 'to': 3, 'and': 4, 'of': 5, 'a': 6, 'i': 7, 'my': 8, 'you': 9, 'that': 10, 'is': 11, 'be': 12, 'not': 13, 'in': 14, 'me': 15, 'for': 16, 'this': 17, "'and": 18, 'your': 19, 'with': 20, 'have': 21, 'it': 22, "'i": 23, 'he': 24, 'thou': 25, 'as': 26, 'but': 27, 'his': 28, 'thy': 29, 'we': 30, 'him': 31, 'her': 32, 'good': 33, 'will': 34, 'no': 35, 'so': 36, 'by': 37, 'all': 38, 'are': 39, "'to": 40, 'what': 41, 'shall': 42, "'the": 43, 'do': 44, "'o": 45, 'thee': 46, 'our': 47, 'sir': 48, 'more': 49, 'on': 50, 'at': 51, 'lord': 52, 'come': 53, 'they': 54, 'she': 55, 'them': 56, 'here': 57, 'now': 58, 'was': 59, 'king': 60, 'then': 61, "'that": 62, 'their': 63, 'or': 64, 'hath': 65, 'how': 66, "'for": 67, 'am': 68, 'yet': 69, 'say': 70, 'us': 71, 'some': 72, "'but": 73, 'had': 74, 'than': 75, "'you": 76, 'would': 77, 'must': 78, "i'll": 79, 'which': 80, 'o': 81, "'my": 82, 'may': 83, 'one': 84, 'if': 85, 'make': 86, 'well': 87, 'go': 88, 'too': 89, 'from': 90, 'father':

In [ ]:
input_sequences = []

for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  # print("LIST = "+str(token_list))
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    # print(n_gram_sequence)
    input_sequences.append(n_gram_sequence)

# print(len(input_sequences))

# #pad sequences
max_sequence_len = max([len(seq) for seq in input_sequences])

print(max_sequence_len, total_words)

input_sequences = np.array(pad_sequences(input_sequences, padding='pre', maxlen=max_sequence_len))

#create predictors and labels
xs, labels = input_sequences[:,:-1], input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

12299 3078


In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(96)))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 12298, 100)        307800    
                                                                 
 bidirectional (Bidirectiona  (None, 12298, 300)       301200    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 12298, 300)        0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 192)              304896    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 1539)              297027    
                                                                 
 dense_1 (Dense)             (None, 3078)              4

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
history = model.fit(xs, ys, epochs=75, verbose=1)

Epoch 1/75
385/385 [==============================] - 610s 2s/step - loss: 6.9823 - acc: 0.0734
Epoch 2/75
 84/385 [=====>........................] - ETA: 7:34 - loss: 6.4228 - acc: 0.0733

KeyboardInterrupt: ignored

In [ ]:
def predict_next_words(seed_text, next_words):
  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predict_x=model.predict(token_list) 
    predicted=np.argmax(predict_x,axis=1)
    # predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seed_text += " " + output_word

  print(seed_text)
  return seed_text

In [ ]:
seed_text = "I am "
next_words = 120


generated_text = predict_next_words(seed_text, next_words)

1/1 [==============================] - 1s 515ms/step
I am  ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
